#### 고려대한국어대사전 오픈프로 포멧으로 변경 (proverb)

1. 오픈프로 필요 기타 칼럼 추가
2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
4. 설명 부분의 < > 부분 용법 내용으로 분류 
5. 품사 폰트 변경
6. "¶" 이걸 기준으로 설명과 예시 분류
7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다.
8. 용례 여러개일 경우 분류
9. 뜻풀이 참조어 링크 태그로 변환
10. 이형태 정보
11. 중복제거
12. 네이버와 아이디 매칭

In [234]:
import pandas as pd
import re
from time import time

start_time = time()

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
symbols = ['①', '②', '③']

# input_file_path 부분을 채워주세요.
input_file_path = "../asset/proverb.csv"
target = pd.read_csv(input_file_path)

In [235]:
# 1. 오픈프로 필요한 기타 칼럼 추가 
target['type'] = '속담'
target['typediff'] = '괄호 없앤 속담/관용구'
target['shoulder'] = int(0)
target['onum'] = int(1)
target = target[['uid', 'eid', 'ohw', 'ocol', 'proverb', 'shoulder',
                'type', 'typediff', 'explan', 'onum']]
target

,uid,eid,ohw,ocol,proverb,shoulder,type,typediff,explan,onum
0,1,120,가갸,가갸뒤자도모른다,【가갸 뒤 자〔뒷다리〕도 모른다】,0,속담,괄호 없앤 속담/관용구,①글자를 모르는 사람을 비유적으로 이르는 말. ②사리에 어두운 사람을 놀림조로 이르...,1
1,2,134,가게,가게기둥에입춘이라,【가게 기둥에 입춘〔주련〕이라】,0,속담,괄호 없앤 속담/관용구,"보잘것없는 가겟집 기둥에 ‘입춘대길(立春大吉)’이라고 써붙인다는 뜻으로, 제격에 맞...",1
2,3,435,가깝다,가까운남이먼친척보다낫다,【가까운 남이 먼 친척〔일가〕보다 낫다】,0,속담,괄호 없앤 속담/관용구,이웃끼리 서로 친하게 지내면 먼 곳에 있는 일가보다 더 친하게 됨을 이르는 말. ¶...,1
3,4,435,가깝다,가까운데를가도점심밥을싸가지고가거라,【가까운 데를 가도 점심밥을 싸 가지고 가거라】,0,속담,괄호 없앤 속담/관용구,무슨 일에나 준비를 든든히 하여 실수가 없게 하라는 말. <유>십 리 길에 점심 싸기.,1
4,5,435,가깝다,가까운데집은깎이고먼데절은비친다,【가까운 데 집은 깎이고 먼 데 절〔집〕은 비친다】,0,속담,괄호 없앤 속담/관용구,가까운 곳에 있는 것은 너무 친숙하여 좋지 않아 보이고 먼 곳에 있는 것은 더 훌륭...,1
...,...,...,...,...,...,...,...,...,...,...
7404,8154,243826,우물,우물안개구리,【우물 안 개구리】,0,속담,괄호 없앤 속담/관용구,"사회의 형편을 모르는, 견문이 좁은 사람. ¶작은 시골 학교에서 1등을 했다고 뻐기...",1
7405,8155,233555,오달지다,오달지기는사돈네가을닭이다,【오달지기는 사돈네 가을 닭이다】,0,속담,괄호 없앤 속담/관용구,"①사돈네 가을 닭이 아무리 살지고 좋기로 제게는 아무 소용이 없다는 뜻으로, 보기만...",1
7406,8156,9710,갗,갗에서좀난다,【갗에서 좀 난다】,0,속담,괄호 없앤 속담/관용구,"①가죽에 좀이 나서 마침내 없어지면 결국 좀도 살 수 없게 된다는 뜻으로, 형제 간...",1
7407,8157,143107,봉산,봉산수숫대같다,【봉산 수숫대 같다】,0,속담,괄호 없앤 속담/관용구,‘키만 멀쑥하게 큰 사람’을 농담으로 이르는 말.,1


In [236]:
# 2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
LEN = len(target)
for i in range(LEN): 
    target.loc[i, 'proverb'] = target.loc[i, 'proverb'].replace('【', '')
    target.loc[i, 'proverb'] = target.loc[i, 'proverb'].replace('】', '')
    target.loc[i, 'proverb'] = target.loc[i, 'proverb'].strip()

In [237]:
# 3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[0] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[1])
        target.loc[i, 'explan'] = temps[0][1:].strip()
        new_row = target.loc[i, :].copy()
        new_row['onum'] = 2
        new_row['explan'] = temps[1].strip()
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[2] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[2])
        target.loc[i, 'explan'] = temps[0].strip()
        new_row = target.loc[i, :].copy()
        new_row['explan'] = temps[1].strip()
        new_row['onum'] = 3
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

In [238]:
# 4. 설명 부분의 < > 부분 용법 내용으로 분류 
target['pattern'] = ''

LEN = len(target)
for i in range(LEN):
    p = r'\s*◀(.*?)▶' 
    m = re.search(p, target.loc[i, 'explan'])
    if m:
        result = m.group().strip() 
        target.loc[i, 'pattern'] = result[1:-1] 
        target.loc[i, 'explan'] = re.sub(p, '', target.loc[i, 'explan']).strip() 

In [239]:
# 5. 품사 폰트 변경 
# 보니까 proverb에는 pattern 정보가 없어 품사 폰트 변경 필요가 없다. 
# 칼럼 순서만 바꿔주자.
target = target[['uid', 'eid', 'ohw', 'ocol', 'proverb', 'shoulder', 'type', 'typediff', 'onum', 'pattern', 'explan']]

In [240]:
# 6. "¶" 이걸 기준으로 설명과 예시 분류
LEN = len(target)
target['explan'] = target['explan'].str.strip()
for i in range(LEN):
    temp = target.loc[i, 'explan'].split('¶')
    target.loc[i, 'explan'] = temp[0].strip()
    if len(temp) != 1:
        target.loc[i, 'example'] = temp[1].strip()
target['example'].fillna('', inplace=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_26120/4163738061.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['example'].fillna('', inplace=True)


In [241]:
# 7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다. (뜻번호당 하나만!!! 용례당 하나가 아님)
target['example'].fillna('', inplace=True)
target['sim_type'] = ''
target['similar_expression'] = ''

LEN = len(target)
for i in range(LEN):
    if '<유>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip() 
        target.loc[i, 'sim_type'] = '유의어'
for i in range(LEN):
    if '<유>' in target.loc[i, 'explan']: 
        temps = target.loc[i, 'explan'].split('<유>')
        target.loc[i, 'explan'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip() 
        target.loc[i, 'sim_type'] = '유의어'

In [242]:
# 8. 용례 여러개일 경우 분류
LEN = len(target)
extra_list = []
for i in range(LEN):
    if '/' in target.loc[i, 'example']:
        temps = target.loc[i, 'example'].split(' / ')
        target.loc[i, 'example'] = temps[0].strip()
        for example in temps[1:]:
            new_row = target.loc[i, :].copy()
            new_row['example'] = temps[1].strip()
            extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'proverb':[extra_list[i]['proverb']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 
                            'explan':[extra_list[i]['explan']], 'example':extra_list[i]['example']})
    target = pd.concat([target, new_row], ignore_index=True)

target['similar_expression'].fillna('', inplace=True)
target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_26120/1615679598.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['similar_expression'].fillna('', inplace=True)


In [243]:
# 9. 뜻풀이 참조어 링크 태그로 변환 + <유>

In [244]:
# 10. 이형태 정보
LEN = len(target)
pattern1 = '\((.*?)\)'
pattern2 = '〔(.*?)〕'
for i in range(LEN):
    if target.loc[i, 'proverb'] != "":
        target.loc[i, 'proverbdiff'] = re.sub(pattern1, "", target.loc[i, 'proverb'])
        target.loc[i, 'proverbdiff'] = re.sub(pattern2, "", target.loc[i, 'proverbdiff'])

target = target[['uid', 'eid', 'ohw', 'ocol', 'proverb', 'proverbdiff', 'shoulder', 'type',
        'typediff', 'onum', 'pattern', 'explan', 'example', 'sim_type',
        'similar_expression']]


In [245]:
# 11. 중복제거
condition = target.duplicated(subset=['uid', 'onum'], keep='first')
condition2 = target.duplicated(subset=['uid'], keep='first')
target.loc[condition, 'uid':'explan'] = ""
target.loc[condition2, 'uid':'typediff'] = ""

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_26120/2700526436.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_26120/2700526436.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_26120/2700526436.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.l

In [246]:
# 12. 네이버와 아이디 매칭
target['naver_id'] = ""
target = target[['uid', 'naver_id', 'eid', 'ohw', 'ocol', 'proverb', 'proverbdiff', 'shoulder', 'type', 'typediff',
                'onum', 'pattern', 'explan', 'example', 'sim_type', 'similar_expression']]

ID = pd.read_csv('../asset/proverb_id.csv')
ID.rename(columns={'naver_ID':'naver_id'}, inplace=True)
ID['is_matched'] = "dismatched"
LEN = len(target)
LEN_ID = len(ID)
for i in range(LEN):
        target_proverb = target.loc[i, 'proverb']
        start = 0
        end = LEN_ID - 1
        while True:
                if start > end:
                        break
                mid = (start + end) // 2
                try_proverb = ID.loc[mid, 'proverb']
                if target_proverb == try_proverb:
                        target.loc[i, 'naver_id'] = ID.loc[mid, 'naver_id']
                        ID.loc[mid, 'is_matched'] = 'matched'
                        break
                elif target_proverb < try_proverb:
                        end = mid - 1
                else:
                        start = mid + 1

In [252]:
condition = (target['naver_id'] == "") & (target['uid'] != "")
# 신규어 개수 (매칭 안된 idiom)
print(f"신규(매칭 안됨) : {len(target[condition])}")


신규(매칭 안됨) : 22


In [253]:
# target
condition2 = ID['is_matched'] == "dismatched"
len(ID[condition2])

23

In [249]:
OHW_ID = pd.read_csv('../asset/ohw_id.csv')
LEN = len(target)
for i in range(LEN):
    if 'EID' in target.loc[i, 'similar_expression']:
        matches = re.findall(r'(\S+)&EID=(\d+);', target.loc[i, 'similar_expression'])
        for word, match in matches:
            matched_int = int(match)
            matched_row = OHW_ID[OHW_ID['kordic_eid'] == matched_int]
            if not matched_row.empty:
                naver_id = matched_row['naver_ID'].iloc[0]
                template = f'<a class="direct" href="https://ko.dict.naver.com/#/entry/koko/{naver_id}">{word}</a>'
                pattern_to_replace = f'{word}&EID={match};'
                target.loc[i, 'similar_expression'] = target.loc[i, 'similar_expression'].replace(pattern_to_replace, template)
            else:
                print("NOT MATCHED")

In [256]:
target.to_csv('../result/processed_proverb.csv', index=False)
end_time = time()
print(f"!!!FINISHED!!! 걸린시간 : {round(end_time - start_time, 2)}s")

!!!FINISHED!!! 걸린시간 : 64.46s


In [254]:
target[condition]

,uid,naver_id,eid,ohw,ocol,proverb,proverbdiff,shoulder,type,typediff,onum,pattern,explan,example,sim_type,similar_expression
846,794,,48467,급하다,급하면바늘허리에실매어쓴다,급하면 바늘허리에 실 매어 쓴다〔쓸까〕,급하면 바늘허리에 실 매어 쓴다,0,속담,괄호 없앤 속담/관용구,1,,"일에는 일정한 순서가 있고 때가 있는 것이므로, 아무리 바빠도 도리에 맞게 순서를 ...",급하다고 바늘허리에 실 매어 쓸까? 그럴수록 침착하게 행동해야 실수가 없는 법이야.,,
955,891,,53230,깎다,깎아놓은밤알같다,깎아 놓은 밤알같다,깎아 놓은 밤알같다,0,속담,괄호 없앤 속담/관용구,1,,생김새나 겉모양이 말쑥하고 깔끔한 사람을 이르는 말.,초등학교 입학식에 갔더니 아이들이 깎아 놓은 밤알같이 다들 예쁘고 의젓해 보이더라.,유의어,깎은 서방님같다.
1006,937,,55842,꾸다,꾸어다놓은보릿자루,꾸어다 놓은 보릿자루〔빗자루〕,꾸어다 놓은 보릿자루,0,속담,괄호 없앤 속담/관용구,1,,여럿이 모여 웃고 떠드는 가운데 혼자 묵묵히 앉아 있는 사람을 비유적으로 이르는 말.,평소에는 말도 많던 네가 왜 오늘은 꾸어다 놓은 보릿자루처럼 조용한 건지 모르겠다.,유의어,전당 잡은 촛대(같고 꾸어 놓은 보릿자루 같다).
1441,1359,,69160,눈,눈가리고아옹한다,눈 가리고 아옹〔아웅〕한다,눈 가리고 아옹한다,0,속담,괄호 없앤 속담/관용구,1,,얕은수로 남을 속이려 한다는 말.,내가 아무리 어리석어도 눈 가리고 아옹하는 수작에는 속지 않아.,,
2292,2187,,100736,마음,마음이흔들비쭉이라,마음이 흔들비쭉이라,마음이 흔들비쭉이라,0,속담,괄호 없앤 속담/관용구,1,,심지(心志)가 굳지 못하고 변덕이 많은 사람을 비유적으로 이르는 말.,,,
3291,3187,,144661,부모,부모속에는부처가들어있고자식속에는앙칼이들어있다,부모 속에는 부처가 들어 있고 자식 속에는 앙칼이 들어 있다,부모 속에는 부처가 들어 있고 자식 속에는 앙칼이 들어 있다,0,속담,괄호 없앤 속담/관용구,1,,부모는 자식에게 모든 것을 희생해 가면서 사랑을 다하지만 자식들 가운데는 부모의 은...,,,
3384,3277,,150503,불타다,불탄개가죽같다,불탄 개가죽 같다,불탄 개가죽 같다,0,속담,괄호 없앤 속담/관용구,1,,일에 발전이 없고 점점 오그라들기만 하는 경우를 비유적으로 이르는 말.,,,
4148,4027,,189159,수박,수박겉핥기,수박 겉 핥기,수박 겉 핥기,0,속담,괄호 없앤 속담/관용구,1,,"수박의 맛있는 속을 먹지 않고 딱딱한 겉만 핥고 있다는 뜻으로, 내용은 모르고 겉만...",수박 겉핥기 식으로 시험공부를 한 경주는 답을 제대로 쓸 수가 없었다.,유의어,"개 머루 먹듯, 꿀단지 겉 핥는다, 언청이 콩가루 먹듯, 후추를 통째로 삼킨다."
4429,4322,,208922,아무리,아무리바빠도바늘허리매어쓰지못한다,아무리 바빠도 바늘허리 매어 쓰지 못한다,아무리 바빠도 바늘허리 매어 쓰지 못한다,0,속담,괄호 없앤 속담/관용구,1,,아무리 바쁘더라도 응당 갖추어야 할 것을 갖추지 않고는 어떤 일을 제대로 할 수 없...,아무리 바빠도 바늘허리 매어 쓰지 못한다고 양말도 안 신고 제사를 지낼 수는 없는 ...,,
4673,4560,,221256,억지로,억지로절받기,억지로 절받기,억지로 절받기,0,속담,괄호 없앤 속담/관용구,1,,"상대방은 절을 하려는 마음도 없는데 이쪽에서 우겨 절을 받는다는 뜻으로, 스스로 무...","칭찬받고 싶어서 한 말은 아닌데, 억지로 절 받는 꼴이 되었네.",유의어,"엎드려 절받기, 옆 찔러 절받기."
